In this assignment, you will be applying the genetic algorithm approach for solving a problem involving locations of healthcare facilities (building on the ideas covered in one of this week's discussion articles and (VIDEO TO COME)).

Mo_pop_Sim.zip Download Mo_pop_Sim.zipcontains simulated data of the residential locations of (assumed) residents of Missouri. And MO_2018_Federally_Qualified_Health_Center_Locations.zip Download MO_2018_Federally_Qualified_Health_Center_Locations.zipcontains data on FQHCs.



Your goal is to identify, from the list of FQHCs (shared as part of this week's uploads), a set of 8 FQHCs where you can provide specialized mental health services. Assume that none of FQHCs present in the data currently have these services, and that you are identifying 8 locations where these services would be introduced for the first time.

How would you determine the relative (i.e., compared to other locations) fitness of a location?



One approach can be to use the average closeness of a location from all of the residences - those locations that are the closest to all of the residences, on average, are the ones that are more easily accessible, and therefore would/should be selected. What is the assumption that is implicit here?
Another approach is to consider locations that have the highest densities of residences within a 30-mile radius (e.g., if location 1 has 59 residences and location 2 has 84 residences in a 30-mile radius, location 1 is to be considered "better" - why? because it can serve more people)
From the above two possibilities, choose one and explain your rationale for choosing it, along with the assumptions you are making in computing the fitness of each location. Then, describe the fitness function - what is the metric that you would compute and would it be maximized or minimized? Why?

Implement a solution using your chosen fitness function and provide details of your solution.

Choose R or Python as your language of choice and create a RMD or IPYNB file that answers the two questions above with the code to back up your assertions. Check your solution into a folder labeled Week06. Submit by entering the URL for your solution.

# **solution**

In this assignment, I am tasked with using a genetic algorithm to select 8 Federally Qualified Health Centers (FQHCs) from a given dataset to introduce specialized mental health services. After considering two possible approaches for determining the fitness of a location, I have chosen to focus on the population density within a 30-mile radius of each FQHC.

Rationale for Choosing Population Density:
Approach Considered:

Average Closeness to All Residents: This approach would select FQHCs based on the average closeness to all residents, assuming that reducing the average distance between residents and FQHCs would improve accessibility. While this approach could optimize geographic distribution, it doesn’t necessarily maximize the number of people served, which is a key factor when expanding access to specialized services like mental health care.

Population Density within 30 Miles: This approach selects FQHCs that are located in areas with the highest number of residents within a 30-mile radius. I chose this approach because it directly aligns with the objective of maximizing the number of individuals who can access mental health services. The assumption here is that by focusing on FQHCs with high population density, we can ensure that a larger number of people will benefit from the new services.

Why I Chose Population Density:

The goal is to maximize the reach and impact of specialized mental health services. Serving a larger population increases the efficiency and effectiveness of healthcare delivery. I believe that selecting locations with higher population density will ensure that we are meeting the needs of the most people.
The fitness calculation, based on the number of residents within a 30-mile radius, assumes that more residents in proximity to an FQHC increases the potential for the FQHC to provide a meaningful impact.

Fitness Function:
The fitness function is designed to evaluate the "fitness" of each selection of 8 FQHC locations. The metric used in this case is the number of residents who are within a 30-mile radius of each selected FQHC. I compute the total population within this radius for each individual in the population (i.e., each selection of FQHCs).

In [7]:
import numpy as np
import pandas as pd
import geopandas as gpd
!pip install deap
from deap import base, creator, tools, algorithms
from scipy.spatial.distance import cdist

# Load Data
fqhc_data = gpd.read_file("MO_2018_Federally_Qualified_Health_Center_Locations.dbf")  # FQHC locations
pop_data = pd.read_csv("Mo_pop_Sim.csv")  # Resident locations

# Extract latitude & longitude
fqhc_locations = fqhc_data[['Latitude', 'Longitude']].values
pop_locations = pop_data[['lat', 'long']].values

def haversine_vectorized(lat1, lon1, lat2_array, lon2_array):
    """Vectorized Haversine function to compute distances in miles for multiple points at once."""
    R = 3959  # Earth radius in miles

    lat1, lon1 = np.radians(lat1), np.radians(lon1)
    lat2_array, lon2_array = np.radians(lat2_array), np.radians(lon2_array)

    dlat = lat2_array - lat1
    dlon = lon2_array - lon1

    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2_array) * np.sin(dlon / 2) ** 2
    return 2 * R * np.arcsin(np.sqrt(a))

# Precompute distances between all FQHCs and residents
all_distances = np.zeros((len(fqhc_locations), len(pop_locations)))
for i, fqhc in enumerate(fqhc_locations):
    all_distances[i] = haversine_vectorized(fqhc[0], fqhc[1], pop_locations[:, 0], pop_locations[:, 1])

# Compute number of residents within 30 miles of an FQHC
def population_within_radius(fqhc_index):
    """Now takes the index of the FQHC in fqhc_locations."""
    distances = all_distances[fqhc_index]  # Use precomputed distances
    return np.sum(distances <= 30)  # Count residents within 30 miles

# Fitness function (maximize population coverage)
def evaluate(individual):
    # Use precomputed distances for efficiency
    return (sum(population_within_radius(fqhc_index) for fqhc_index in individual),)

# Genetic Algorithm setup
creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # Maximize fitness
creator.create("Individual", list, fitness=creator.FitnessMax)

# Create unique individual (no duplicates in FQHC selections)
def unique_individual():
    """Create an individual with unique FQHCs (no repetition)."""
    return list(np.random.choice(len(fqhc_locations), 8, replace=False))

toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, unique_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Run Genetic Algorithm
population = toolbox.population(n=20)
algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=5, verbose=True)

# Get best solution
best_individual = tools.selBest(population, k=1)[0]
best_fqhcs = fqhc_locations[np.array(best_individual)]

# Print Selected FQHC Locations
print("Selected FQHC Locations for Mental Health Services:")
print(best_fqhcs)


/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals
0  	20    
1  	11    
2  	10    
3  	15    
4  	10    
5  	10    
Selected FQHC Locations for Mental Health Services:
[[ 39.083164 -94.507583]
 [ 38.66863  -90.272661]
 [ 38.435946 -90.554678]
 [ 38.668384 -90.209452]
 [ 38.677759 -90.230247]
 [ 37.241458 -90.968494]
 [ 39.257031 -94.451666]
 [ 39.035322 -94.539588]]


** Description of the Solution**

In the code, I implemented a genetic algorithm to find the best 8 FQHCs based on the fitness function described above. Below is a breakdown of the steps I took in the code:

**Data Loading:**

I loaded two datasets: one for FQHC locations (latitude and longitude) and one for the simulated population data (residential locations).

**Distance Calculation (Haversine Formula):**

I used a vectorized version of the Haversine formula to calculate the distances between each FQHC and each resident. This formula computes the great-circle distance between two points on the Earth’s surface, which is necessary for evaluating the fitness of each location.

**Population Coverage Calculation: **

I created a function to calculate how many residents are within a 30-mile radius of each FQHC. This step used the precomputed distance matrix from the Haversine formula.

**Genetic Algorithm Setup:**

I set up the genetic algorithm using the DEAP library. The goal of the algorithm is to select a set of 8 FQHCs that maximize the number of residents served. The individuals in the population represent different selections of 8 FQHCs, and the fitness of each individual is determined by the sum of the number of residents within 30 miles of each selected FQHC.

**Ensuring Uniqueness:**

I added a constraint to ensure that no duplicates are selected within a given individual. This guarantees that the selected FQHCs are all distinct.

**Algorithm Execution:**

I ran the genetic algorithm for 5 generations, with crossover and mutation rates defined, and used a tournament selection method to pick the best individuals for the next generation.

**Final Selection:**

After running the genetic algorithm, I selected the best individual and output the corresponding FQHC locations, which are the optimal locations for providing specialized mental health services.

Result:
The genetic algorithm produced the following selected FQHC locations for the provision of specialized mental health services:

[[ 39.083164 -94.507583]

 [ 38.66863  -90.272661]

 [ 38.435946 -90.554678]

 [ 38.668384 -90.209452]

 [ 38.677759 -90.230247]

 [ 37.241458 -90.968494]

 [ 39.257031 -94.451666]

 [ 39.035322 -94.539588]]


These are the 8 FQHCs that have been chosen based on the fitness function, which maximizes the number of residents that fall within a 30-mile radius of the selected locations. These FQHCs represent the optimal set of locations for providing the new mental health services in Missouri.

Conclusion:

In conclusion, I chose the approach based on population density within a 30-mile radius of FQHCs because it aligns with the goal of maximizing the number of people who can access specialized mental health services. The fitness function was designed to maximize the total number of residents within the 30-mile radius of the selected FQHCs, ensuring that the chosen locations serve the largest population possible. The genetic algorithm was implemented to select the best 8 FQHCs, and the final result provides a set of locations that should be ideal for introducing mental health services.

In [ ]:
#@title Convert ipynb to HTML in Colab
# Upload ipynb
from google.colab import files
f = files.upload()

# Convert ipynb to html
import subprocess
file0 = list(f.keys())[0]
_ = subprocess.run(["pip", "install", "nbconvert"])
_ = subprocess.run(["jupyter", "nbconvert", file0, "--to", "html"])

# download the html
files.download(file0[:-5]+"html")